In [1]:
from keras.applications import ResNet50, VGG16
from keras.models import Sequential
from keras.layers import Dense, AveragePooling2D
import pandas as pd
import cv2
import numpy as np
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import train_test_split, GridSearchCV
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout,AveragePooling2D
from keras.preprocessing.image import ImageDataGenerator

In [2]:
df = pd.read_csv('../train.csv')

In [3]:
b_size = 128
num_classes = 4
ep = 3

In [4]:
l1=[]
l2=[]
# пробегаемся по каждому столбцу датасета
for img,ClassId,EncodedPixels in tqdm(df.values):
    try:
      # считываем изображения из папки
      image=cv2.imread("../train_images/{}".format(img),cv2.IMREAD_COLOR)
      # меняем размер на 120х120
      image = cv2.resize(image,(224,224))
      # разделяем по спискам
      l1.append(image)
      l2.append(ClassId)
    except:
      continue

100%|██████████| 7095/7095 [00:06<00:00, 1082.13it/s]


In [5]:
encoder = LabelEncoder()

x_image = np.array(l1) / 255

y_label = np.array(encoder.fit_transform(l2))

In [6]:
reshaped_X = x_image.reshape(x_image.shape[0],-1)

oversample = RandomOverSampler(sampling_strategy='not majority')
x_image, y_label  = oversample.fit_resample(reshaped_X , y_label)
x_image = x_image.reshape(-1,224,224,3)

In [7]:
y_label = to_categorical(y_label)

In [8]:
train_image, test_image, train_label, test_label = train_test_split(x_image, y_label, test_size=0.2, stratify=y_label, shuffle=True)

In [9]:
datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = datagen.flow(train_image, train_label, batch_size=b_size)
test_generator = datagen.flow(test_image, test_label, batch_size=b_size)

In [10]:
base_model = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3))

In [11]:
model_4 = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(4, activation='softmax')
])

model_4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [12]:
# %%time

# es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

# history_4 = model_4.fit(train_image,train_label, epochs=5, validation_data = (test_image, test_label), batch_size=128, verbose=1, callbacks=[es_callback] )
# if es_callback.stopped_epoch >=1:
#     print(f'Обучение остановлено на {es_callback.stopped_epoch} эпохе')
# test_loss, test_acc = model_4.evaluate(test_image, test_label, verbose=2)
# print(f'Точность модели model_4: {100*test_acc:.2f} %')
# res_model[f'model_4'] = history_4

In [13]:
model_4.fit_generator(train_generator,
                    steps_per_epoch=len(train_image) // b_size,
                    epochs=ep,
                    validation_data=test_generator,
                    validation_steps=len(test_image) // b_size)

# Оценка производительности модели на тестовых данных
score = model_4.evaluate_generator(test_generator, steps=len(test_image) // b_size)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6260\3544681814.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_4.fit_generator(train_generator,


Epoch 1/3


: 